# Análisis Exploratorio de Datos - Predicción de Contratación

Este notebook contiene el análisis exploratorio de los datos de postulaciones para entrenar el modelo de predicción de contratación.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Configurar estilo de gráficos
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = [12, 8]

## 1. Carga de Datos

In [ ]:
# Cargar datos
df = pd.read_csv('../postulaciones_sinteticas_500.csv')
print(f"Forma del dataset: {df.shape}")
print(f"\nPrimeras 5 filas:")
df.head()

In [ ]:
# Información general del dataset
print("Información del dataset:")
df.info()
print("\nEstadísticas descriptivas:")
df.describe()

## 2. Análisis de la Variable Objetivo

In [ ]:
# Crear variable objetivo desde 'estado'
target_mapping = {
    'aceptado': 1,
    'entrevista': 1, 
    'contratado': 1,
    'rechazado': 0,
    'en revisión': 0,
    'pendiente': 0
}

df['contactado'] = df['estado'].str.lower().map(target_mapping).fillna(0)

# Distribución de la variable objetivo
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Gráfico de barras
target_counts = df['contactado'].value_counts()
axes[0].bar(['No Contactado', 'Contactado'], target_counts.values)
axes[0].set_title('Distribución de Variable Objetivo')
axes[0].set_ylabel('Cantidad')

# Gráfico de pie
axes[1].pie(target_counts.values, labels=['No Contactado', 'Contactado'], autopct='%1.1f%%')
axes[1].set_title('Proporción de Contactados')

plt.tight_layout()
plt.show()

print(f"Distribución de la variable objetivo:")
print(f"No contactados: {target_counts[0]} ({target_counts[0]/len(df)*100:.1f}%)")
print(f"Contactados: {target_counts[1]} ({target_counts[1]/len(df)*100:.1f}%)")

## 3. Análisis de Variables Numéricas

In [ ]:
# Convertir variables numéricas
df['años_experiencia'] = pd.to_numeric(df['años_experiencia'], errors='coerce')
df['salario'] = pd.to_numeric(df['salario'], errors='coerce')

# Análisis de años de experiencia
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Distribución de años de experiencia
axes[0, 0].hist(df['años_experiencia'].dropna(), bins=20, edgecolor='black')
axes[0, 0].set_title('Distribución de Años de Experiencia')
axes[0, 0].set_xlabel('Años de Experiencia')
axes[0, 0].set_ylabel('Frecuencia')

# Años de experiencia por resultado
df.boxplot(column='años_experiencia', by='contactado', ax=axes[0, 1])
axes[0, 1].set_title('Años de Experiencia por Resultado')
axes[0, 1].set_xlabel('Contactado (0=No, 1=Sí)')

# Distribución de salarios
axes[1, 0].hist(df['salario'].dropna(), bins=20, edgecolor='black')
axes[1, 0].set_title('Distribución de Salarios')
axes[1, 0].set_xlabel('Salario')
axes[1, 0].set_ylabel('Frecuencia')

# Salario por resultado
df.boxplot(column='salario', by='contactado', ax=axes[1, 1])
axes[1, 1].set_title('Salario por Resultado')
axes[1, 1].set_xlabel('Contactado (0=No, 1=Sí)')

plt.tight_layout()
plt.show()

## 4. Análisis de Variables Categóricas

In [ ]:
# Análisis de nivel educativo
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Distribución de nivel educativo
education_counts = df['nivel_educacion'].value_counts()
axes[0, 0].bar(education_counts.index, education_counts.values)
axes[0, 0].set_title('Distribución de Nivel Educativo')
axes[0, 0].tick_params(axis='x', rotation=45)

# Tasa de contacto por nivel educativo
education_contact_rate = df.groupby('nivel_educacion')['contactado'].mean()
axes[0, 1].bar(education_contact_rate.index, education_contact_rate.values)
axes[0, 1].set_title('Tasa de Contacto por Nivel Educativo')
axes[0, 1].set_ylabel('Tasa de Contacto')
axes[0, 1].tick_params(axis='x', rotation=45)

# Distribución de industrias
industry_counts = df['industria'].value_counts().head(10)
axes[1, 0].barh(industry_counts.index, industry_counts.values)
axes[1, 0].set_title('Top 10 Industrias')

# Tasa de contacto por industria
industry_contact_rate = df.groupby('industria')['contactado'].mean().sort_values(ascending=False).head(10)
axes[1, 1].barh(industry_contact_rate.index, industry_contact_rate.values)
axes[1, 1].set_title('Tasa de Contacto por Industria (Top 10)')
axes[1, 1].set_xlabel('Tasa de Contacto')

plt.tight_layout()
plt.show()

## 5. Análisis de Habilidades

In [ ]:
# Análisis de habilidades más comunes
all_skills = []
for skills_str in df['habilidades'].dropna():
    skills = [skill.strip().lower() for skill in str(skills_str).split(',')]
    all_skills.extend(skills)

from collections import Counter
skill_counts = Counter(all_skills)
top_skills = dict(skill_counts.most_common(15))

plt.figure(figsize=(12, 8))
plt.barh(list(top_skills.keys()), list(top_skills.values()))
plt.title('Top 15 Habilidades Más Mencionadas')
plt.xlabel('Frecuencia')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

print(f"Total de habilidades únicas: {len(skill_counts)}")
print(f"Top 5 habilidades: {list(top_skills.keys())[:5]}")

## 6. Análisis Temporal

In [ ]:
# Convertir fechas
df['fecha_postulacion'] = pd.to_datetime(df['fecha_postulacion'], errors='coerce')
df['fecha_publicacion'] = pd.to_datetime(df['fecha_publicacion'], errors='coerce')

# Crear features temporales
df['dias_desde_publicacion'] = (df['fecha_postulacion'] - df['fecha_publicacion']).dt.days
df['mes_postulacion'] = df['fecha_postulacion'].dt.month
df['dia_semana'] = df['fecha_postulacion'].dt.dayofweek

fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Postulaciones por mes
monthly_applications = df['mes_postulacion'].value_counts().sort_index()
axes[0, 0].plot(monthly_applications.index, monthly_applications.values, marker='o')
axes[0, 0].set_title('Postulaciones por Mes')
axes[0, 0].set_xlabel('Mes')
axes[0, 0].set_ylabel('Cantidad')

# Tasa de contacto por mes
monthly_contact_rate = df.groupby('mes_postulacion')['contactado'].mean()
axes[0, 1].plot(monthly_contact_rate.index, monthly_contact_rate.values, marker='o', color='orange')
axes[0, 1].set_title('Tasa de Contacto por Mes')
axes[0, 1].set_xlabel('Mes')
axes[0, 1].set_ylabel('Tasa de Contacto')

# Distribución de días desde publicación
axes[1, 0].hist(df['dias_desde_publicacion'].dropna(), bins=20, edgecolor='black')
axes[1, 0].set_title('Distribución de Días desde Publicación')
axes[1, 0].set_xlabel('Días')
axes[1, 0].set_ylabel('Frecuencia')

# Tasa de contacto por día de la semana
weekday_names = ['Lun', 'Mar', 'Mié', 'Jue', 'Vie', 'Sáb', 'Dom']
weekday_contact_rate = df.groupby('dia_semana')['contactado'].mean()
axes[1, 1].bar(range(7), weekday_contact_rate.values)
axes[1, 1].set_title('Tasa de Contacto por Día de la Semana')
axes[1, 1].set_xlabel('Día de la Semana')
axes[1, 1].set_ylabel('Tasa de Contacto')
axes[1, 1].set_xticks(range(7))
axes[1, 1].set_xticklabels(weekday_names)

plt.tight_layout()
plt.show()

## 7. Correlaciones

In [ ]:
# Matriz de correlación para variables numéricas
numeric_cols = ['años_experiencia', 'salario', 'dias_desde_publicacion', 'contactado']
correlation_matrix = df[numeric_cols].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, 
            square=True, linewidths=0.5)
plt.title('Matriz de Correlación')
plt.tight_layout()
plt.show()

print("Correlaciones con la variable objetivo:")
target_correlations = correlation_matrix['contactado'].drop('contactado').sort_values(key=abs, ascending=False)
for var, corr in target_correlations.items():
    print(f"{var}: {corr:.3f}")

## 8. Insights y Conclusiones

In [ ]:
print("=== INSIGHTS DEL ANÁLISIS EXPLORATORIO ===")
print()

# Balance de clases
positive_rate = df['contactado'].mean()
print(f"1. BALANCE DE CLASES:")
print(f"   - Tasa de contacto general: {positive_rate:.1%}")
if positive_rate < 0.3 or positive_rate > 0.7:
    print(f"   - ⚠️  Dataset desbalanceado - considerar técnicas de rebalanceo")
else:
    print(f"   - ✅ Balance aceptable para entrenamiento")

print()

# Variables más importantes
print(f"2. VARIABLES MÁS CORRELACIONADAS:")
for var, corr in target_correlations.head(3).items():
    direction = "positiva" if corr > 0 else "negativa"
    print(f"   - {var}: {corr:.3f} (correlación {direction})")

print()

# Calidad de datos
print(f"3. CALIDAD DE DATOS:")
missing_pct = (df.isnull().sum() / len(df) * 100).sort_values(ascending=False)
for col, pct in missing_pct.head(5).items():
    if pct > 0:
        print(f"   - {col}: {pct:.1f}% valores faltantes")

print()

# Recomendaciones para el modelo
print(f"4. RECOMENDACIONES PARA EL MODELO:")
print(f"   - ✅ Incluir feature engineering para habilidades (coincidencias)")
print(f"   - ✅ Considerar variables temporales (mes, día semana)")
print(f"   - ✅ Usar TF-IDF para campos de texto")
print(f"   - ✅ Normalizar variables numéricas")
print(f"   - ✅ Encoding adecuado para variables categóricas")

if positive_rate < 0.3:
    print(f"   - ⚠️  Considerar técnicas de rebalanceo (SMOTE, class weights)")
    print(f"   - ⚠️  Usar métricas apropiadas (ROC AUC, PR AUC)")

print()
print("=== ANÁLISIS COMPLETADO ===")